# Testing model building with `carveme` and `memote`

Importing the COBRA module to read sbml models created via the `carveme` method. Subsequently we reconstruct a draft of the GSM model of *S. cerevisae* using the found RefSeq ID. Looking at the [reference genome R64](https://www.ncbi.nlm.nih.gov/data-hub/taxonomy/4932/?utm_source=assembly&utm_medium=referral&utm_campaign=KnownItemSensor:taxname), we see that 6464 have been annotated.  

In [8]:
%%capture
from cobra.io import read_sbml_model
!carve --refseq GCF_000146045.2_R64 --output Saccharomyces.xml

In [9]:
print("DONE WITH THE CARVING!")

DONE WITH THE CARVING!


In [10]:
model = read_sbml_model('Saccharomyces.xml')

In [11]:
print("GSM model fact sheet:")
print("Number of reactions:", len(model.reactions))
print("Number of metabolites:", len(model.metabolites))
print("Number of genes included in the model:", len(model.genes))
print("% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes):",
      round((len(model.genes)/6464)*100, 2))
print("Gene coverage percentage =", round(len(model.genes)/len(model.reactions)*100, 2), "%")

GSM model fact sheet:
Number of reactions: 1007
Number of metabolites: 754
Number of genes included in the model: 537
% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes): 8.31
Gene coverage percentage = 53.33 %


In [12]:
!memote report snapshot --skip test_stoichiometric_consistency \
    --skip test_find_metabolites_not_produced_with_open_bounds \
    --skip test_find_metabolites_not_consumed_with_open_bounds Saccharomyces.xml --filename Saccharomyces.html

============================= test session starts ==============================
platform linux -- Python 3.6.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /usr/local/lib/python3.6/dist-packages/memote/suite/tests
collected 146 items / 1 skipped / 145 selected                                 

../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_annotation.py F [  0%]
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF..         [ 44%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F.F...FF                                                   [ 60%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_biomass.py . [ 60%]
F.....F.F                                                                [ 67%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_consistency.py s [ 67%]
FFssssssssssssssssFFFF.F.ss.                                             [ 86%]

In [13]:
!memote report snapshot Saccharomyces.xml --filename Saccharomyces.html

============================= test session starts ==============================
platform linux -- Python 3.6.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /usr/local/lib/python3.6/dist-packages/memote/suite/tests
collected 146 items / 1 skipped / 145 selected                                 

../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_annotation.py F [  0%]
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF..         [ 44%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F.F...FF                                                   [ 60%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_biomass.py . [ 60%]
F.....F.F                                                                [ 67%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_consistency.py F [ 67%]
FFssssssssssssssssFFFF.F.FF.                                             [ 86%]